In [1]:
! pip install "autogen-ext[openai]"
! pip install autogen-agentchat
! pip install autogen-core
! pip install ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.4/331.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, b

In [2]:
import os
import sqlite3
import json
from datetime import datetime
from typing import Literal, Annotated

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.conditions import FunctionalTermination
from autogen_agentchat.messages import BaseChatMessage
from typing import Sequence, Any
from autogen_core.tools import FunctionTool
from termcolor import colored, cprint
from ddgs import DDGS
from typing import Annotated
from datetime import datetime
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"]=secret_value_0


In [3]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini",temperature=0.7)

In [4]:
if os.path.exists("flight_deals.db"):
    os.remove("flight_deals.db")

conn = sqlite3.connect("flight_deals.db")
c = conn.cursor()
c.execute("""CREATE TABLE deals (
    city TEXT PRIMARY KEY,
    country TEXT,
    round_trip_price REAL,
    currency TEXT DEFAULT 'USD',
    last_updated TEXT
)""")
conn.commit()

deals = [
    ("London", "UK", 298, "2025-11-20"),
    ("Paris", "France", 379, "2025-11-20"),
    ("Tokyo", "Japan", 689, "2025-11-19"),
    ("Dubai", "UAE", 499, "2025-11-20"),
    ("Bali (Denpasar)", "Indonesia", 789, "2025-11-18"),
    ("New York", "USA", 198, "2025-11-20"),
    ("Sydney", "Australia", 998, "2025-11-19"),
    ("Cancun", "Mexico", 289, "2025-11-20"),
]

c.executemany("INSERT INTO deals VALUES (?, ?, ?, 'USD', ?)", deals)
conn.commit()
conn.close()


In [5]:
def get_flight_price(city: Annotated[str, "Destination city"], departure_city: Annotated[str, "Departure city"] = "New York") -> str:
    conn = sqlite3.connect("flight_deals.db")
    c = conn.cursor()
    c.execute("SELECT city, country, round_trip_price FROM deals WHERE city LIKE ?", (f"%{city}%",))
    rows = c.fetchall()
    conn.close()
    if not rows:
        return f"No deals found for {city}."
    lines = [f"{city_name}, {country} → ${price} round-trip" for city_name, country, price in rows[:3]]
    return "\n".join(lines) + "\n\nBook at skydeal.ai (demo)"

def visa_check(destination: Annotated[str, "Destination"]) -> str:
    return f"Quick visa info for {destination}: Most nationalities get visa-free or visa-on-arrival. Check official sources."


def web_search(query: Annotated[str, "Search query"]) -> str:
    current_month_year = datetime.now().strftime("%B %Y")
    timely_query = f"{query} {current_month_year}"
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(timely_query, max_results=6))
        if not results:
            return f"No live results for '{query}' in {current_month_year}."
        
        lines = [f"Live results for '{query}' ({current_month_year}):"]
        
        for r in results[:4]:
            title = r["title"]
            body = r["body"][:130].replace("\n", " ")
            link = r["href"]
            lines.append(f"• {title}\n  {body}...\n  → {link}\n")
        
        lines.append("Compare live prices: google.com/travel/flights")
        return "\n".join(lines)
    
    except Exception as e:
        return f"Search failed: {e}. Using database."


flight_tool = FunctionTool(get_flight_price, description="Get flight prices from database")
visa_tool = FunctionTool(visa_check, description="Check visa requirements")
search_tool = FunctionTool(web_search, description="Real-time web search for flight deals")

In [6]:
smart_travel_agent = AssistantAgent(
    name="SkyDeal_AI",
    model_client=model_client,
    system_message="""
You are SkyDeal AI — the fastest, wittiest, brutally honest flight deal hunter on Earth.

NON-NEGOTIABLE RULES (you will be fired if you break them):
1. ALWAYS use web_search first for any "latest", "right now", "current", or "today" request.
2. For NYC → Bali round-trip in 2025: REAL price range is $847–$980. Anything under $800 is fake, one-way, or scam.
   → If web_search shows $300–$600 prices, you MUST say: "Those are one-way clickbait ads. Real round-trip starts at $847."
3. Never trust DDG/KAYAK headlines blindly — they are 90% one-way or expired.
4. Cross-check every price with common sense + your internal knowledge.
5. Current real deals (November 2025):
   - Cheapest: $847 Philippine Airlines
   - Best value: $850 Turkish Airlines
   - Premium: $905 Cathay Pacific
6. Always quote ROUND-TRIP prices only.
7. Include airline name + exact price + one fun emoji.
8. Be short (2–4 sentences max), sassy, confident, and zero fluff.
9. When user wants to book → end exactly with: "Ready to lock it in? Handing off to booking team..."

Perfect example response:
"Web is screaming $418 but that's one-way garbage. Real round-trip NYC–Bali right now: $847 on Philippine Airlines or $850 Turkish. Still a steal for paradise. Ready to lock it in? Handing off to booking team..."

You are not allowed to be nice about fake prices. Roast them.
You are not allowed to be long-winded. Speed is your brand.
You are SkyDeal AI — the only travel agent that never lies.
""",
    tools=[search_tool, flight_tool, visa_tool],
    reflect_on_tool_use=True
)

In [7]:
booking_manager = AssistantAgent(
    name="Booking_Manager",
    model_client=model_client,
    system_message="""
    You are the final booking agent.
    Your ONLY job is to confirm the booking in ONE short, fun message.
    Rules — you MUST follow all of them:
    - Always give fake confirmation code: SKY42069
    - Always end your message with exactly this tag on its own line: [BOOKING_DONE]
    - Never ask questions
    - Never say anything after the tag
    - Example response:
      "Boom! Your flight is locked in! Confirmation: SKY42069 ✈️\n[BOOKING_DONE]"
    
    Do not write anything else.
    """,
    tools=[]
)

def stop_condition(events):
    text = " ".join(str(getattr(e, "content", "")) for e in events)
    return "[BOOKING_DONE]" in text or len(events) > 12

travel_team = RoundRobinGroupChat(
    participants=[smart_travel_agent, booking_manager],
    termination_condition=FunctionalTermination(stop_condition),
    max_turns=4
)


In [8]:
async def run_travel_team():
    print(colored("SkyDeal AI — Live Flight Deal Hunter", "cyan", attrs=["bold"]))
    print(colored("═" * 70, "cyan"))
    task = "What's the cheapest flight to Bali from NYC right now? Search the web!"
    cprint(f"\nYou asked → {task}\n", "white", "on_grey", attrs=["bold"])
    print(colored("AI is thinking…\n", "yellow"))

    result = await travel_team.run(task=task)

    print(colored("Live Conversation", "green", attrs=["bold", "underline"]))
    print("─" * 70)
    for msg in result.messages:
        if not hasattr(msg, "source"):
            continue
        src = msg.source
        content = str(msg.content) if msg.content else ""
        if src == "user":
            cprint(f"You: {content}", "white", "on_blue", attrs=["bold"])
        elif src == "SkyDeal_AI":
            cprint(f"SkyDeal AI: {content}", "cyan", attrs=["bold"])
        elif src == "Booking_Manager":
            cprint(f"Booking Agent: {content}", "magenta", attrs=["bold"])
        elif "FunctionCall" in str(type(msg)):
            cprint("Tool Used: Searching web / database…", "yellow")

    if "[BOOKING_DONE]" in "".join(str(m.content) for m in result.messages if hasattr(m, "content")):
        cprint("\nBooking Confirmed!", "white", "on_green", attrs=["bold"])
        cprint("Confirmation Code: SKY42069", "green", attrs=["bold"])
    print(colored("═" * 70, "cyan"))

await run_travel_team()

SkyDeal AI — Live Flight Deal Hunter
══════════════════════════════════════════════════════════════════════

You asked → What's the cheapest flight to Bali from NYC right now? Search the web!

AI is thinking…

Live Conversation
──────────────────────────────────────────────────────────────────────
You: What's the cheapest flight to Bali from NYC right now? Search the web!
SkyDeal AI: [FunctionCall(id='call_3XmELVgKXzgPJFAfbfiti36h', arguments='{"query":"cheap flights NYC to Bali November 2025"}', name='web_search')]
SkyDeal AI: [FunctionExecutionResult(content="Live results for 'cheap flights NYC to Bali November 2025' (November 2025):\n• Flight Deal: U.S. to Bali from $560 Round-Trip | Condé Nast\n  ... November and then again between January through March 2019, including Thanksgiving, according to Scott s Cheap Flights , who f...\n  → https://www.cntraveler.com/story/us-to-bali-from-dollar560-round-trip\n\n• Cheap flights to Bali in Winter! Only €479! - RushFlights.com\n  Check cheap